# User environment settings

In [6]:
# Standard library imports
import platform
import os
from IPython.display import clear_output
from pathlib import Path

# Local imports
import BiblioAnalysis_Utils as bau

clear_output(wait=True)

# Set the venv use status
venv = False
print('Virtual environment: ', venv)

# Get the information of current operating system
os_name = platform.uname().system
print('Operating system:    ', os_name)
if os_name=='Darwin':bau.add_site_packages_path(venv)

# User identification
user_root = Path.home()
user_id =  str(user_root)[str(user_root).rfind('/')+1:]
print('User:                ', user_id)
expert =  False

# Select the corpuses folder
corpuses_folder = bau.select_folder_gui(user_root,'Select the corpuses folder')
print('\nCorpuses folder:', corpuses_folder)

Virtual environment:  False
Operating system:     Windows
User:                 C:\Users\ld259969

Corpuses folder: C:/Users/ld259969/Documents/PyVenv/BiblioMeterDraft/Liten_Corpuses


## I - Selection

In [18]:
# Standard library imports
import os
from pathlib import Path

# Local imports
import BiblioAnalysis_Utils as bau

## Selection of corpus file
corpusfiles_list = os.listdir(corpuses_folder)
corpusfiles_list.sort()
print('Please select the corpus via the tk window')
myprojectname = bau.Select_multi_items(corpusfiles_list,'single')[0]+'/'
project_folder = corpuses_folder /Path(myprojectname)
database_type = input('Corpus file type (scopus, wos - default: "wos")? ')
if database_type =='': database_type = 'wos' 

rep_utils =''
if database_type =='scopus':
     # Get the folder for the general files
     # and specific files for scopus type database in this folder
    if os_name=='Darwin':
        rep_utils = os.path.abspath('BiblioAnalysis_RefFiles')        
    else:
        rep_utils = bau.select_folder_gui(user_root,'Select the folder specific files for scopus')        
    scopus_cat_codes = bau.SCOPUS_CAT_CODES
    scopus_journals_issn_cat = bau.SCOPUS_JOURNALS_ISSN_CAT
    print('Folder of the specific files for scopus:', rep_utils)       
    
## Setting the  graph main heading
digits_list = list(filter(str.isdigit, myprojectname))
corpus_year = ''
for i in range(len(digits_list)):corpus_year = corpus_year + digits_list[i]
init = str(corpuses_folder).rfind("_")+1
corpus_state = str(corpuses_folder)[init:]
main_heading = corpus_year + ' Corpus: ' + corpus_state

## Printing useful information
print('\nSpecific-paths set for user: ', user_id)
print('Project folder:              ', project_folder)
print('Corpus year:                 ', corpus_year)
print('Corpus status:               ', corpus_state)
print('Project name:                ', myprojectname)
print('Corpus file type:            ', database_type)

Please select the corpus via the tk window


Corpus file type (scopus, wos - default: "wos")?  wos



Specific-paths set for user:  C:\Users\ld259969
Project folder:               C:\Users\ld259969\Documents\PyVenv\BiblioMeterDraft\Liten_Corpuses\2021_wos
Corpus year:                  2021
Corpus status:                Corpuses
Project name:                 2021_wos/
Corpus file type:             wos


## II Data Parsing

In [19]:
# Standard libraries import
import os
import json
from pathlib import Path

# Local imports
import BiblioAnalysis_Utils as bau

## Building the names of the useful folders

    # Folder containing the wos or scopus file to process
in_dir_parsing = project_folder / Path(bau.FOLDER_NAMES['rawdata'])

    # Folder containing the output files of the data parsing 
out_dir_parsing = project_folder / Path(bau.FOLDER_NAMES['parsing'])
if not os.path.exists(out_dir_parsing):
    os.mkdir(out_dir_parsing)

## Running function biblio_parser
parser_done = input("Parsing available (y/n)? ")
if parser_done == "n":
     # Setting the specific affiliations filter (default = None)
    second_inst = input("Secondary institutions to be parsed (y/n)? ")
    if second_inst=='y' : 
        inst_filter_dic= {'secondary_inst': ['LITEN','INES'],
                          'country': 'France'} 
    else:
        inst_filter_dic = None
    bau.biblio_parser(in_dir_parsing, out_dir_parsing, database_type, expert, rep_utils, inst_filter_dic) 
    with open(Path(out_dir_parsing) / Path('failed.json'), 'r') as failed_json:
            data_failed=failed_json.read()
    dic_failed = json.loads(data_failed)
    articles_number = dic_failed["number of article"]
    print("Parsing processed on full corpus")
    print("\n\nSuccess rates")
    del dic_failed['number of article']
    for item, value in dic_failed.items():
        print(f'    {item}: {value["success (%)"]:.2f}%')
else:
    parser_filt = input("Parsing available without rawdata -from filtering- (y/n)? ")
    if parser_filt == "n":        
        with open(Path(out_dir_parsing) / Path('failed.json'), 'r') as failed_json:
            data_failed=failed_json.read()
        dic_failed = json.loads(data_failed)
        articles_number = dic_failed["number of article"]
        #clear_output(wait=True)
        print("Parsing available from full corpus")
        print("\n\nSuccess rates")
        del dic_failed['number of article']
        for item, value in dic_failed.items():
            print(f'    {item}: {value["success (%)"]:.2f}%')
    else:
        #clear_output(wait=True)
        print("Parsing available from filtered corpus without rawdata")
        file = project_folder /Path('parsing/' + 'articles.dat')
        with open(file) as f:
            lines = f.readlines()
        articles_number = len(lines)

print("\n\nCorpus parsing saved in folder:\n", str(out_dir_parsing))
print('\nNumber of articles in the corpus : ', articles_number)

Parsing available (y/n)?  n
Secondary institutions to be parsed (y/n)?  y


Parsing processed on full corpus


Success rates
    AK: 69.18%
    IK: 77.36%
    TK: 100.00%
    Address: 100.00%
    Country: 100.00%
    Institution: 100.00%
    authors_inst: 100.00%
    Subject: 100.00%
    Sub_subject: 100.00%


Corpus parsing saved in folder:
 C:\Users\ld259969\Documents\PyVenv\BiblioMeterDraft\Liten_Corpuses\2021_wos\parsing

Number of articles in the corpus :  159
